Source: https://github.com/iamtrask/Grokking-Deep-Learning/blob/master/README.md 

        Chapter11 - Intro to Word Embeddings - Neural Networks that Understand Language.ipynb 



23 min for two epochs (as defined below); you can't use GPU nor TPU;  

In [1]:
from datetime import datetime # for the sake of measuring real ellapsed time


# Download the IMDB Dataset

In [2]:
# Download reviews.txt and labels.txt from here: https://github.com/udacity/deep-learning/tree/master/sentiment-network


In [3]:
!wget "https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/reviews.txt"


--2020-10-22 01:22:00--  https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33678267 (32M) [text/plain]
Saving to: ‘reviews.txt’

reviews.txt         100%[===================>]  32.12M  57.6MB/s    in 0.6s    

2020-10-22 01:22:01 (57.6 MB/s) - ‘reviews.txt’ saved [33678267/33678267]



In [4]:
!cat reviews.txt | head -30

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

In [5]:
!wget "https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/labels.txt"
!cat labels.txt | head -30

--2020-10-22 01:22:02--  https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/labels.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225000 (220K) [text/plain]
Saving to: ‘labels.txt’

labels.txt          100%[===================>] 219.73K  --.-KB/s    in 0.04s   

2020-10-22 01:22:02 (5.15 MB/s) - ‘labels.txt’ saved [225000/225000]

positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative


In [6]:
def pretty_print_review_and_label(i):
   print(labels[i] + "\t:\t" + reviews[i][:80] + "...")

g = open('reviews.txt','r') # What we know!
reviews = list(map(lambda x:x[:-1],g.readlines()))
g.close()

g = open('labels.txt','r') # What we WANT to know!
labels = list(map(lambda x:x[:-1].upper(),g.readlines()))
g.close()

# Capturing Word Correlation in Input Data

In [7]:
import numpy as np

onehots = {}
onehots['cat'] = np.array([1,0,0,0])
onehots['the'] = np.array([0,1,0,0])
onehots['dog'] = np.array([0,0,1,0])
onehots['sat'] = np.array([0,0,0,1])

sentence = ['the','cat','sat']
x = onehots[sentence[0]] + \
    onehots[sentence[1]] + \
    onehots[sentence[2]]

print("Sent Encoding:" + str(x))

Sent Encoding:[1 1 0 1]


# Predicting Movie Reviews

In [8]:
import sys
f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

f = open('labels.txt')
raw_labels = f.readlines()
f.close()


In [9]:
#preparing the data
tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))

vocab = set()
for sent in tokens:
    for word in sent:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

target_dataset = list()
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

In [10]:
print(len(vocab))

74074


In [11]:
import numpy as np
np.random.seed(1)

def sigmoid(x):
    return 1/(1 + np.exp(-x))


hidden_size = 100

weights_0_1 = 0.2*np.random.random((len(vocab),hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,1)) - 0.1

print (weights_0_1)

[[-0.0165956   0.0440649  -0.09997713 ...  0.0147359  -0.09942593
   0.02342898]
 [-0.03467102  0.00541162  0.07718842 ... -0.05312758  0.02335567
   0.08980326]
 [ 0.09003522  0.01133064  0.08312127 ... -0.08407478  0.09656342
  -0.06367743]
 ...
 [-0.07656205 -0.09023205 -0.00978559 ... -0.0008865  -0.04352412
  -0.06427378]
 [-0.08444666 -0.07143386 -0.05731608 ... -0.06584719 -0.02770722
   0.09327983]
 [ 0.03695404 -0.05106593  0.02970002 ...  0.0043325   0.00801544
  -0.09468272]]




correct,total = (0,0)

for iter in range(iterations):
    
    # train on first 24,000
    for i in range(len(input_dataset)-1000):

        x,y = (input_dataset[i],target_dataset[i])
        layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0)) #embed + sigmoid
        layer_2 = sigmoid(np.dot(layer_1,weights_1_2)) # linear + softmax

        layer_2_delta = layer_2 - y # compare pred with truth
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) #backprop

        weights_0_1[x] -= layer_1_delta * alpha
        weights_1_2 -= np.outer(layer_1,layer_2_delta) * alpha

        if(np.abs(layer_2_delta) < 0.5):
            correct += 1
        total += 1
        if(i % 10 == 9):
            progress = str(i/float(len(input_dataset)))
            sys.stdout.write('\rIter:'+str(iter)\
                             +' Progress:'+progress[2:4]\
                             +'.'+progress[4:6]\
                             +'% Training Accuracy:'\
                             + str(correct/float(total)) + '%')
    print()
    
correct,total = (0,0)
for i in range(len(input_dataset)-1000,len(input_dataset)):

    x = input_dataset[i]
    y = target_dataset[i]

    layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
    
    if(np.abs(layer_2 - y) < 0.5):
        correct += 1
    total += 1
print("Test Accuracy:" + str(correct / float(total)))




In [12]:
start=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
start

'2020-10-22 01:22:07'

In [13]:
tokens[0]

{'',
 '\n',
 '.',
 'a',
 'about',
 'adults',
 'age',
 'all',
 'and',
 'as',
 'at',
 'believe',
 'bromwell',
 'burn',
 'can',
 'cartoon',
 'classic',
 'closer',
 'comedy',
 'down',
 'episode',
 'expect',
 'far',
 'fetched',
 'financially',
 'here',
 'high',
 'i',
 'immediately',
 'in',
 'insightful',
 'inspector',
 'is',
 'isn',
 'it',
 'knew',
 'lead',
 'life',
 'line',
 'm',
 'many',
 'me',
 'much',
 'my',
 'of',
 'one',
 'other',
 'pathetic',
 'pettiness',
 'pity',
 'pomp',
 'profession',
 'programs',
 'ran',
 'reality',
 'recalled',
 'remind',
 'repeatedly',
 'right',
 's',
 'sack',
 'same',
 'satire',
 'saw',
 'school',
 'schools',
 'scramble',
 'see',
 'situation',
 'some',
 'student',
 'students',
 'such',
 'survive',
 't',
 'teachers',
 'teaching',
 'than',
 'that',
 'the',
 'their',
 'think',
 'through',
 'time',
 'to',
 'tried',
 'welcome',
 'what',
 'when',
 'which',
 'who',
 'whole',
 'years',
 'your'}

# Comparing Word Embeddings

In [14]:
from collections import Counter
import math 

def similar(target='beautiful'):
    target_index = word2index[target]
    scores = Counter()
    for word,index in word2index.items():
        raw_difference = weights_0_1[index] - (weights_0_1[target_index])
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))

    return scores.most_common(10)

In [15]:
beatiful_before = similar('beautiful')
print(beatiful_before)

[('beautiful', -0.0), ('horowitz', -0.6279863871181349), ('detachment', -0.6338550248516636), ('paperweight', -0.6376699585602446), ('debralee', -0.6388730094567298), ('anywhoo', -0.6391596331234375), ('gingernuts', -0.6392911580299072), ('evocation', -0.6402499083184466), ('decreed', -0.6404874987583421), ('hooves', -0.6413835699304542)]


In [16]:
terrible_before = similar('terrible')
print(similar('terrible'))

[('terrible', -0.0), ('gordons', -0.6098341418793334), ('nichol', -0.6167738301464554), ('rackham', -0.616852047634792), ('channeled', -0.6230779404189136), ('airwolfs', -0.6252340878326009), ('womennone', -0.6282976987693409), ('hellhole', -0.6298910707597197), ('tempered', -0.6361710254207527), ('askew', -0.6365074824289562)]


# Filling in the Blank

In [17]:
import sys,random,math
from collections import Counter
import numpy as np

np.random.seed(1)
random.seed(1)
f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

tokens = list(map(lambda x:(x.split(" ")),raw_reviews))
wordcnt = Counter()
for sent in tokens:
    for word in sent:
        wordcnt[word] -= 1
vocab = list(set(map(lambda x:x[0],wordcnt.most_common())))

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

concatenated = list()
input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
            concatenated.append(word2index[word])
        except:
            ""
    input_dataset.append(sent_indices)
concatenated = np.array(concatenated)
random.shuffle(input_dataset)

In [26]:
epochs =10


alpha, iterations = (0.05, epochs)
hidden_size,window,negative = (50,2,5)  

weights_0_1 = (np.random.rand(len(vocab),hidden_size) - 0.5) * 0.2
weights_1_2 = np.random.rand(len(vocab),hidden_size)*0

layer_2_target = np.zeros(negative+1)
layer_2_target[0] = 1

def similar(target='beautiful'):
  target_index = word2index[target]

  scores = Counter()
  for word,index in word2index.items():
    raw_difference = weights_0_1[index] - (weights_0_1[target_index])
    squared_difference = raw_difference * raw_difference
    scores[word] = -math.sqrt(sum(squared_difference))
  return scores.most_common(10)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

for rev_i,review in enumerate(input_dataset * iterations):
  for target_i in range(len(review)):
        
    # since it's really expensive to predict every vocabulary
    # we're only going to predict a random subset
    target_samples = [review[target_i]]+list(concatenated\
    [(np.random.rand(negative)*len(concatenated)).astype('int').tolist()])

    left_context = review[max(0,target_i-window):target_i]
    right_context = review[target_i+1:min(len(review),target_i+window)]

    layer_1 = np.mean(weights_0_1[left_context+right_context],axis=0)
    layer_2 = sigmoid(layer_1.dot(weights_1_2[target_samples].T))
    layer_2_delta = layer_2 - layer_2_target
    layer_1_delta = layer_2_delta.dot(weights_1_2[target_samples])

    weights_0_1[left_context+right_context] -= layer_1_delta * alpha
    weights_1_2[target_samples] -= np.outer(layer_2_delta,layer_1)*alpha

  if(rev_i % 1000 == 0):
    sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)
        *iterations)) + "   " + str(similar('terrible')))
    #sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)*iterations)))
    #print(similar('terrible'))

Progress:0.996   [('terrible', -0.0), ('horrible', -2.652617087846785), ('dreadful', -3.3399194007678883), ('pathetic', -3.6477505444295653), ('lousy', -3.6746369292792624), ('brilliant', -3.773535400600973), ('lame', -3.7831401223720746), ('marvelous', -3.801721081463413), ('baaad', -3.94324134815086), ('bad', -3.9447589901217768)]

In [28]:
terrible_after=similar('terrible')
beautiful_after=similar('beautiful')

print(terrible_after)
print(terrible_before)

print()

print(beautiful_after)
print(beatiful_before)

[('terrible', -0.0), ('horrible', -3.047108766131703), ('bad', -3.3483678206129666), ('pathetic', -3.362994159837182), ('dreadful', -3.430916233442514), ('poor', -3.826789171438953), ('brilliant', -3.868361153447606), ('lousy', -3.928901046022573), ('lame', -3.9331143832691513), ('mediocre', -3.9669875206659007)]
[('terrible', -0.0), ('gordons', -0.6098341418793334), ('nichol', -0.6167738301464554), ('rackham', -0.616852047634792), ('channeled', -0.6230779404189136), ('airwolfs', -0.6252340878326009), ('womennone', -0.6282976987693409), ('hellhole', -0.6298910707597197), ('tempered', -0.6361710254207527), ('askew', -0.6365074824289562)]

[('beautiful', -0.0), ('lovely', -3.6072511284587288), ('cute', -3.81349315032832), ('creepy', -3.8645696297042376), ('charming', -4.162778988422358), ('gorgeous', -4.20144141354433), ('frightening', -4.4228969910519895), ('ditzy', -4.49551519675573), ('cool', -4.536256047251313), ('weepy', -4.539795944644388)]
[('beautiful', -0.0), ('horowitz', -0.627

# King - Man + Woman ~= Queen

In [29]:
def analogy(positive=['terrible','good'],negative=['bad']):
    
    norms = np.sum(weights_0_1 * weights_0_1,axis=1)
    norms.resize(norms.shape[0],1)
    
    normed_weights = weights_0_1 * norms
    
    query_vect = np.zeros(len(weights_0_1[0]))
    for word in positive:
        query_vect += normed_weights[word2index[word]]
    for word in negative:
        query_vect -= normed_weights[word2index[word]]
    
    scores = Counter()
    for word,index in word2index.items():
        raw_difference = weights_0_1[index] - query_vect
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))
        
    return scores.most_common(10)[1:]

In [30]:
analogy(['terrible','good'],['bad'])

[('stellar', -180.73171960395854),
 ('terrific', -180.7843373923732),
 ('rewarded', -180.94544602172962),
 ('tremendous', -180.96896994763318),
 ('decent', -181.02320285144165),
 ('worth', -181.3311579496993),
 ('great', -181.36497573536033),
 ('superb', -181.40123274532803),
 ('legitimate', -181.51038203157947)]

[('stellar', -180.73171960395854),
 ('terrific', -180.7843373923732),
 ('rewarded', -180.94544602172962),
 ('tremendous', -180.96896994763318),
 ('decent', -181.02320285144165),
 ('worth', -181.3311579496993),
 ('great', -181.36497573536033),
 ('superb', -181.40123274532803),
 ('legitimate', -181.51038203157947)]

**Note** Try other analogies. Note how the vectors are obtained: they are sampled from positive and separately from negative reviews. Therefore analogy(_) is defined with these particular parameters. 

In [1]:
analogy(['he','brother'],['she'])

NameError: ignored

[('brother', -360.5061551100639),
 ('sister', -361.690544670385),
 ('br', -361.7046180354739),
 ('wife', -361.7397943535972),
 ('father', -361.8696239485474),
 ('friend', -362.038248899882),
 ('mother', -362.1219000841776),
 ('daughter', -362.1933177120553),
 ('dad', -362.36320983425964)]

In [27]:
end = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
end 

'2020-10-22 04:28:44'